In [1]:
import argparse
import logging
import math
import os
import random
import sys
import time
from copy import deepcopy
from pathlib import Path
import pandas as pd
import matplotlib.pylab as plt
import numpy as np
import cv2 as cv
import argparse
import pafy
from pytube import YouTube

In [2]:
#from pytube import YouTube
#YouTube('https://www.youtube.com/watch?v=Hr5dMZgNj4g').streams.first().download()

In [3]:
#BODY_PARTS = { "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
#               "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
#               "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
#               "LEye": 15, "REar": 16, "LEar": 17, "Background": 18 }


#POSE_PAIRS = [ ["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
#               ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
#               ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
#               ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"], ["Nose", "REye"],
#               ["REye", "REar"], ["Nose", "LEye"], ["LEye", "LEar"] ]

BODY_PARTS = {"Head": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
                "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
                "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "Chest": 14,
                "Background": 15 }

POSE_PAIRS = [ ["Head", "Neck"], ["Neck", "RShoulder"], ["RShoulder", "RElbow"],
             ["RElbow", "RWrist"], ["Neck", "LShoulder"], ["LShoulder", "LElbow"],
             ["LElbow", "LWrist"], ["Neck", "Chest"], ["Chest", "RHip"], ["RHip", "RKnee"],
             ["RKnee", "RAnkle"], ["Chest", "LHip"], ["LHip", "LKnee"], ["LKnee", "LAnkle"] ]

In [4]:
inWidth = 500 #368
inHeight = 500  #368

In [5]:
net = cv.dnn.readNetFromTensorflow("graph_opt.pb")  #create custom
#net = cv.dnn.readNetFromTensorflow("kapao_l_coco.pt")

In [6]:
#cap = cv.VideoCapture("train/Tom Brady 2000 NFL Scouting Combine highlights.mp4")
#cap = cv.VideoCapture("train/Herbert_JimmyG.mp4")
cap = cv.VideoCapture("bryce_young.gif")

In [7]:
hasFrame,frame = cap.read()

In [9]:
#frame

In [10]:
count = 0
while cv.waitKey(1) < 0:
    hasFrame, frame = cap.read()
    if not hasFrame:
        #cv.waitKey()
        cv.destroyAllWindows()
        break

    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]
    
    #net.setInput(cv.dnn.blobFromImage(frame, 1.0, (inWidth, inHeight), (127.5, 127.5, 127.5), swapRB=True, crop=False))
    net.setInput(cv.dnn.blobFromImage(frame, 1.0, (inWidth, inHeight), swapRB=True, crop=False))
    out = net.forward()
    out = out[:, :16, :, :]  # MobileNet output [1, 57, -1, -1], we only need the first 19 elements
    ### original 19 shapes

    assert(len(BODY_PARTS) == out.shape[1])
    
    points = []
    point_df = pd.DataFrame([])
    frame_lst = []
    column_names = ["frame"]
    frame_df = pd.DataFrame(columns = column_names)
    for i in range(len(BODY_PARTS)):
        # Slice heatmap of corresponging body's part.
        heatMap = out[0, i, :, :]
        #count += 1
        # Originally, we try to find all the local maximums. To simplify a sample
        # we just find a global one. However only a single pose at the same time
        # could be detected this way.
        _, conf, _, point = cv.minMaxLoc(heatMap)
        x = (frameWidth * point[0]) / out.shape[3]
        y = (frameHeight * point[1]) / out.shape[2]
        # Add a point if it's confidence is higher than threshold.
        points.append((int(x), int(y)) if conf > 0.2 else None)
        frame_lst.append((count) if conf > 0.2 else None)
        point_df = point_df.append(points)
        frame_df = frame_df.append(frame_lst)
        #count += 1
        #frame_df.loc[frame_df['frame'] == count]



    for pair in POSE_PAIRS:
        partFrom = pair[0]
        partTo = pair[1]
        assert(partFrom in BODY_PARTS)
        assert(partTo in BODY_PARTS)

        idFrom = BODY_PARTS[partFrom]
        idTo = BODY_PARTS[partTo]

        if points[idFrom] and points[idTo]:
            cv.line(frame, points[idFrom], points[idTo], (255, 74, 0), 3)
            cv.ellipse(frame, points[idFrom], (4, 4), 0, 0, 360, (255, 255, 255), cv.FILLED)
            cv.ellipse(frame, points[idTo], (4, 4), 0, 0, 360, (255, 255, 255), cv.FILLED)
            cv.putText(frame, str(idFrom), points[idFrom], cv.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255),2,cv.LINE_AA)
            cv.putText(frame, str(idTo), points[idTo], cv.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255),2,cv.LINE_AA)
            
            
            #cv.line(frame, points[idFrom], points[idTo], (0, 255, 0), 3)
            #cv.ellipse(frame, points[idFrom], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)
            #cv.ellipse(frame, points[idTo], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)

    t, _ = net.getPerfProfile()
    freq = cv.getTickFrequency() / 1000
    cv.putText(frame, '%.2fms' % (t / freq), (10, 20), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))
    #cv.imwrite("output/frame%d.jpg" % count, frame)
    count = count + 1
    cv.imshow('OpenPose using OpenCV', frame)
    
cv.destroyAllWindows()  

In [11]:
pd.options.display.max_rows = 4000
point_df

,0
0,None
0,None
1,None
0,None
1,None
2,None
0,None
1,None
2,None
3,None


In [12]:
frame_df

,frame,0
0,NaN,NaN
0,NaN,NaN
1,NaN,NaN
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN


In [ ]:
### create frame index
### split x, y coordinates -> 

In [ ]:
import seaborn as sns
x = points_objects[0]
y = points_objects[1]
points = pd.DataFrame(x,y)